In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests
import json
import pandas as pd
from pandas import DataFrame, Series


In [2]:
all_seasons = []

for season in range(2021, 2024):
    summary_url = f"https://api.nhle.com/stats/rest/en/skater/summary?limit=-1&cayenneExp=seasonId={season}{season+1}%20and%20gameTypeId=2"

    try:
        summary_resp = requests.get(summary_url)
        summary_resp.raise_for_status() 
        summary_json =  summary_resp.json()

        if summary_json['data']:
            df_summary = DataFrame(summary_json['data'])
            all_seasons.append(df_summary)
            df_summary['season'] = f"{season}-{season + 1}"
            print(f"Successfully fetched data for season {season}-{season+1}")
        else:
            print(f"No data returned for season {season}-{season + 1}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for season {season}-{season + 1}: {e}")

if all_seasons:
    nhl_api_df = pd.concat(all_seasons, ignore_index=True)
    nhl_api_df = nhl_api_df.groupby('playerId').agg({
            'playerId': 'first',
            'skaterFullName': 'first',
            'positionCode': 'first',
            'gamesPlayed': 'sum',
            'otGoals': 'sum',
            'gameWinningGoals': 'sum',
        }).reset_index(drop = True)
    
print(nhl_api_df)

Successfully fetched data for season 2021-2022
Successfully fetched data for season 2022-2023
Successfully fetched data for season 2023-2024
      playerId     skaterFullName positionCode  gamesPlayed  otGoals  \
0      8465009        Zdeno Chara            D           72        0   
1      8466138       Joe Thornton            C           34        0   
2      8469455       Jason Spezza            C           71        0   
3      8470281       Duncan Keith            D           64        0   
4      8470595         Eric Staal            C           72        0   
...        ...                ...          ...          ...      ...   
1250   8484314       Jiri Smejkal            L           20        0   
1251   8484321  Nikolas Matinpalo            D            4        0   
1252   8484325    Waltteri Merela            C           19        0   
1253   8484326        Patrik Koch            D            1        0   
1254   8484911        Collin Graf            R            7        

In [3]:
nhl_api_df = nhl_api_df.loc[(nhl_api_df['positionCode'] != 'D') & (nhl_api_df['gamesPlayed'] >= 60)]
nhl_api_df = nhl_api_df.reset_index(drop = True)
nhl_api_df = nhl_api_df.fillna(0)

nhl_api_df.rename(columns = {'gameWinningGoals': 'game_winning_goals'}, inplace = True)
nhl_api_df.rename(columns = {'otGoals': 'ot_goals'}, inplace = True)
nhl_api_df.rename(columns = {'skaterFullName': 'Player'}, inplace = True)

In [4]:
goals_up_one_df = pd.read_excel("C:\\Users\\shakj\\Desktop\\NHL-Clutch-Goalscorers\\Goals Up 1.xlsx")
goals_down_one_df = pd.read_excel("C:\\Users\\shakj\\Desktop\\NHL-Clutch-Goalscorers\\Goals Down 1.xlsx")
goals_tied_df = pd.read_excel("C:\\Users\\shakj\\Desktop\\NHL-Clutch-Goalscorers\\Goals Tied.xlsx")
totals_df = pd.read_excel("C:\\Users\\shakj\\Desktop\\NHL-Clutch-Goalscorers\\Totals.xlsx")

In [5]:
goals_up_one_df.rename(columns = {'Goals': 'goals_up_by_one'}, inplace = True)
goals_down_one_df.rename(columns = {'Goals': 'goals_down_by_one'}, inplace = True)
goals_tied_df.rename(columns = {'Goals': 'goals_when_tied'}, inplace = True)
totals_df.rename(columns = {'Goals': 'total_goals'}, inplace = True)

In [6]:
merged = pd.merge(pd.merge(goals_up_one_df[['Player', 'GP', 'goals_up_by_one']],goals_down_one_df[['Player', 'goals_down_by_one']],on='Player'),goals_tied_df[['Player', 'goals_when_tied']],on='Player')
merged = merged.loc[merged['GP'] >= 60]

In [7]:
merged

,Player,GP,goals_up_by_one,goals_down_by_one,goals_when_tied
0,Auston Matthews,228,32,30,56
1,David Pastrnak,236,31,18,65
2,Sebastian Aho,232,31,15,40
3,Zach Hyman,235,31,18,28
4,Chris Kreider,242,31,19,36
...,...,...,...,...,...
728,Elmer Soderblom,21,0,0,3
733,Mitchell Chaffee,32,0,0,2
767,Ivan Miroshnichenko,21,0,0,0
772,Milos Kelemen,24,0,0,0


In [8]:
nhl_api_df

,playerId,Player,positionCode,gamesPlayed,ot_goals,game_winning_goals
0,8469455,Jason Spezza,C,71,0,1
1,8470595,Eric Staal,C,72,0,1
2,8470604,Jeff Carter,C,227,2,8
3,8470606,Dustin Brown,R,64,0,0
4,8470610,Zach Parise,L,194,1,7
...,...,...,...,...,...,...
508,8483515,Juraj Slafkovsky,L,121,0,2
509,8483570,Ben Meyers,C,67,0,0
510,8483808,Andrei Kuzmenko,L,153,3,8
511,8484144,Connor Bedard,C,68,1,2


In [9]:
nhl_api_df.merge(merged, on = 'Player', how = 'left')

,playerId,Player,positionCode,gamesPlayed,ot_goals,game_winning_goals,GP,goals_up_by_one,goals_down_by_one,goals_when_tied
0,8469455,Jason Spezza,C,71,0,1,71.0,2.0,4.0,1.0
1,8470595,Eric Staal,C,72,0,1,72.0,3.0,2.0,3.0
2,8470604,Jeff Carter,C,227,2,8,227.0,12.0,10.0,9.0
3,8470606,Dustin Brown,R,64,0,0,64.0,1.0,2.0,2.0
4,8470610,Zach Parise,L,194,1,7,194.0,3.0,6.0,12.0
...,...,...,...,...,...,...,...,...,...,...
508,8483515,Juraj Slafkovsky,L,121,0,2,121.0,2.0,5.0,6.0
509,8483570,Ben Meyers,C,67,0,0,67.0,0.0,1.0,2.0
510,8483808,Andrei Kuzmenko,L,153,3,8,153.0,8.0,13.0,18.0
511,8484144,Connor Bedard,C,68,1,2,68.0,2.0,6.0,9.0
